# Analyse du Marché des Emplois Tech
## Projet  - Session 1

Ce notebook vous guide à travers le processus de scraping et d'analyse du marché de l'emploi tech.

**Objectifs :**
- Scraper des données d'emplois tech depuis des sites spécialisés
- Extraire des informations clés : titre, entreprise, localisation, type de contrat, niveau d'expérience
- Analyser les technologies recherchées et les tendances du marché
- Sauvegarder les données pour visualisation

**Focus de ce projet :**
- Analyse des types de contrats (Remote/Hybrid/On-site)
- Niveau d'expérience requis
- Stack technique recherchée
- Tendances géographiques


## 1. Imports et Configuration


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from collections import Counter


In [5]:
# Configuration
REQUEST_DELAY = 2  # Délai en secondes entre les requêtes
MAX_JOBS_TO_SCRAPE = 50  # Limite pour la démonstration

# Headers pour simuler un navigateur
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7'
}


# Recupérer la liste de l'URL des emploies

## 3. Fonctions Utilitaires pour l'Extraction

Ces fonctions extraient des informations spécifiques depuis les descriptions d'emploi.


In [6]:
def detect_work_mode(text):
    """
    Détecte le type de contrat (Remote/Hybrid/On-site) depuis le texte
    """
    if not text:
        return "Non spécifié"
    
    text_lower = text.lower()
    
    # Mots-clés pour remote
    remote_keywords = ['remote', 'télétravail', 'work from home', 'wfh', 'fully remote', '100% remote']
    # Mots-clés pour hybrid
    hybrid_keywords = ['hybrid', 'hybride', 'partially remote', 'flexible', '2-3 days']
    # Mots-clés pour on-site
    onsite_keywords = ['on-site', 'on site', 'onsite', 'office', 'bureau', 'présentiel']
    
    remote_count = sum(1 for keyword in remote_keywords if keyword in text_lower)
    hybrid_count = sum(1 for keyword in hybrid_keywords if keyword in text_lower)
    onsite_count = sum(1 for keyword in onsite_keywords if keyword in text_lower)
    
    if remote_count > 0 and remote_count >= hybrid_count:
        return "Remote"
    elif hybrid_count > 0:
        return "Hybrid"
    elif onsite_count > 0:
        return "On-site"
    else:
        return "Non spécifié"

def extract_experience_level(text):
    """
    Extrait le niveau d'expérience requis depuis le texte
    """
    if not text:
        return "Non spécifié"
    
    text_lower = text.lower()
    
    # Patterns pour différents niveaux
    patterns = {
        "Junior": [r'junior', r'entry level', r'0-2 years', r'1-2 years', r'debutant'],
        "Mid-level": [r'mid-level', r'mid level', r'2-5 years', r'3-5 years', r'intermediate'],
        "Senior": [r'senior', r'5\+ years', r'5+ years', r'experienced', r'expérimenté'],
        "Lead/Principal": [r'lead', r'principal', r'staff', r'architect', r'10\+ years']
    }
    
    for level, pattern_list in patterns.items():
        for pattern in pattern_list:
            if re.search(pattern, text_lower):
                return level
    
    return "Non spécifié"

def extract_tech_stack(text):
    """
    Extrait les technologies mentionnées dans la description
    """
    if not text:
        return []
    
    # Liste de technologies courantes
    tech_keywords = {
        'Python', 'JavaScript', 'Java', 'TypeScript', 'Go', 'Rust', 'C++', 'C#',
        'React', 'Vue', 'Angular', 'Node.js', 'Django', 'Flask', 'FastAPI',
        'AWS', 'Azure', 'GCP', 'Docker', 'Kubernetes', 'Terraform',
        'PostgreSQL', 'MongoDB', 'MySQL', 'Redis', 'Elasticsearch',
        'Git', 'CI/CD', 'Jenkins', 'GitHub Actions',
        'Machine Learning', 'TensorFlow', 'PyTorch', 'Scikit-learn',
        'GraphQL', 'REST API', 'Microservices', 'Kafka', 'RabbitMQ'
    }
    
    found_techs = []
    text_lower = text.lower()
    
    for tech in tech_keywords:
        # Recherche insensible à la casse avec word boundaries
        pattern = r'\b' + re.escape(tech.lower()) + r'\b'
        if re.search(pattern, text_lower, re.IGNORECASE):
            found_techs.append(tech)
    
    return found_techs

def extract_salary_range(text):
    """
    Extrait la fourchette salariale depuis le texte
    """
    if not text:
        return None, None
    
    # Patterns pour trouver les salaires (ex: $80k-$120k, €50,000-€70,000)
    patterns = [
        r'\$?(\d+)[kK]?\s*-\s*\$?(\d+)[kK]?',  # $80k-$120k ou 80k-120k
        r'€?(\d+)[,.]?\d*\s*-\s*€?(\d+)[,.]?\d*',  # €50,000-€70,000
        r'(\d+)\s*to\s*(\d+)\s*k',  # 80 to 120k
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            try:
                min_sal = int(match.group(1).replace(',', '').replace('.', ''))
                max_sal = int(match.group(2).replace(',', '').replace('.', ''))
                # Convertir en milliers si nécessaire
                if min_sal < 1000:
                    min_sal *= 1000
                if max_sal < 1000:
                    max_sal *= 1000
                return min_sal, max_sal
            except:
                continue
    
    return None, None


## 4. Fonction Principale d'Extraction

Cette fonction extrait toutes les données d'une page d'emploi.


In [7]:
def extract_job_details_from_aijobs(url, soup=None):
    """
    Extrait les détails complets d'une offre d'emploi depuis aijobs.ai
    
    Cette fonction utilise nos fonctions utilitaires pour détecter automatiquement
    le type de contrat, le niveau d'expérience et les technologies recherchées.
    """
    if soup is None:
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Erreur lors de la récupération de {url}: {e}")
            return None
    
    job_data = {
        'job_title': None,
        'company_name': None,
        'location': None,
        'work_mode': None,  # Remote/Hybrid/On-site (détecté automatiquement)
        'experience_level': None,  # Junior/Mid/Senior (détecté automatiquement)
        'salary_min': None,
        'salary_max': None,
        'tech_stack': [],  # Liste des technologies (détectées automatiquement)
        'job_description': None,
        'job_type': None,  # Full Time, Part Time, etc.
        'job_url': url
    }
    
    # Extraction du titre depuis aijobs.ai
    title_elem = soup.find("div", class_="post-main-title2")
    if title_elem:
        job_data['job_title'] = title_elem.get_text(strip=True)
    
    # Extraction de l'entreprise depuis aijobs.ai
    # Méthode 1 : Chercher le span avec "at"
    company_elem = soup.find("span", string=lambda x: x and "at" in str(x).lower())
    if company_elem:
        company_span = company_elem.find_next_sibling("span")
        if company_span:
            job_data['company_name'] = company_span.get_text(strip=True)
    
    # Méthode 2 : Chercher le lien de l'entreprise
    if not job_data['company_name']:
        company_link = soup.find("a", href=re.compile(r"/company/"))
        if company_link:
            company_name_elem = company_link.find("span", class_="tw-card-title")
            if company_name_elem:
                job_data['company_name'] = company_name_elem.get_text(strip=True)
    
    # Extraction du type d'emploi (Full Time, etc.)
    job_type_elem = soup.find("span", class_=re.compile(r"tw-bg-\[#0BA02C\]"))
    if job_type_elem:
        job_data['job_type'] = job_type_elem.get_text(strip=True)
    
    # Extraction de la localisation depuis aijobs.ai
    location_elem = soup.find("div", class_="remote")
    if location_elem:
        location_p = location_elem.find("p", class_="tw-mb-0")
        if location_p:
            job_data['location'] = location_p.get_text(strip=True)
    
    # Extraction de la description complète
    desc_container = soup.find("div", class_="job-description-container")
    if desc_container:
        description_text = desc_container.get_text(separator=' ', strip=True)
    else:
        # Fallback : prendre tout le body
        body = soup.find('body')
        if body:
            description_text = body.get_text(separator=' ', strip=True)
        else:
            description_text = ""
    
    job_data['job_description'] = description_text[:5000]  # Limiter la taille
    
    # 🎯 UTILISER NOS FONCTIONS UTILITAIRES pour détecter automatiquement :
    # C'est ce qui différencie ce projet - nous analysons le texte pour extraire des infos
    
    # Détecter le type de contrat (Remote/Hybrid/On-site)
    job_data['work_mode'] = detect_work_mode(description_text)
    
    # Détecter le niveau d'expérience requis
    job_data['experience_level'] = extract_experience_level(description_text)
    
    # Extraire les technologies recherchées
    job_data['tech_stack'] = extract_tech_stack(description_text)
    
    # Extraire la fourchette salariale
    min_sal, max_sal = extract_salary_range(description_text)
    job_data['salary_min'] = min_sal
    job_data['salary_max'] = max_sal
    
    # Si pas de salaire trouvé dans la description, chercher dans les éléments spécifiques
    if not min_sal and not max_sal:
        # Chercher dans les sections de salaire
        salary_section = soup.find("div", string=re.compile(r"Salary", re.IGNORECASE))
        if salary_section:
            salary_text = salary_section.get_text()
            min_sal, max_sal = extract_salary_range(salary_text)
            job_data['salary_min'] = min_sal
            job_data['salary_max'] = max_sal
    
    return job_data


## 5. Scraping des URLs d'Emplois depuis Plusieurs Pays

Nous allons scraper depuis aijobs.ai pour plusieurs pays :
- 🇺🇸 États-Unis
- 🇬🇧 Royaume-Uni  
- 🇨🇦 Canada

Les résultats de tous les pays seront combinés pour une analyse globale du marché.

**Note :** Cette section montre comment collecter les URLs d'emplois depuis plusieurs localisations.
Vous devrez adapter les sélecteurs CSS selon le site que vous utilisez.


In [8]:
def collect_job_urls_from_aijobs(max_pages=5, location="United%20States"):
    """
    Collecte les URLs des offres d'emploi depuis aijobs.ai
    
    Cette fonction utilise les sélecteurs spécifiques d'aijobs.ai
    """
    BASE_URL = "https://aijobs.ai"
    job_urls = []
    
    for page_num in range(1, max_pages + 1):
        try:
            # Construire l'URL de la page
            url = f"{BASE_URL}/engineer?location={location}&page={page_num}"
            
            print(f"📄 Scraping page {page_num}...")
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Trouver toutes les cartes d'emploi (sélecteur spécifique d'aijobs.ai)
            job_cards = soup.find_all("a", class_="jobcardStyle1")
            
            # Extraire les URLs
            page_job_urls = []
            for card in job_cards:
                href = card.get("href")
                if href:
                    # Convertir en URL absolue si nécessaire
                    if href.startswith('/'):
                        from urllib.parse import urljoin
                        href = urljoin(BASE_URL, href)
                    if href not in page_job_urls:
                        page_job_urls.append(href)
            
            job_urls.extend(page_job_urls)
            print(f"  ✅ {len(page_job_urls)} emplois trouvés sur cette page")
            
            # Délai entre les requêtes pour respecter le serveur
            time.sleep(REQUEST_DELAY)
            
        except Exception as e:
            print(f"❌ Erreur sur la page {page_num}: {e}")
            continue
    
    # Supprimer les doublons
    unique_urls = list(set(job_urls))
    print(f"\n✅ Total: {len(unique_urls)} URLs d'emplois uniques collectées")
    return unique_urls

# Configuration des pays à scraper
COUNTRIES_TO_SCRAPE = {
    "🇺🇸 États-Unis": "United%20States",
    "🇬🇧 Royaume-Uni": "United%20Kingdom",
    "🇨🇦 Canada": "Canada"
}

# Collecter les URLs d'emplois depuis tous les pays
print("🔍 Collecte des URLs d'emplois depuis plusieurs pays...\n")
print(f"Pays à scraper: {', '.join(COUNTRIES_TO_SCRAPE.keys())}\n")

all_job_urls = {}
total_urls = 0

for country_name, location_code in COUNTRIES_TO_SCRAPE.items():
    print(f"\n{'='*60}")
    print(f"🌍 {country_name}")
    print(f"{'='*60}")
    
    job_urls = collect_job_urls_from_aijobs(max_pages=2, location=location_code)
    all_job_urls[country_name] = job_urls
    total_urls += len(job_urls)
    print(f"✅ {len(job_urls)} URLs collectées pour {country_name}")

print(f"\n{'='*60}")
print(f"✅ TOTAL: {total_urls} URLs d'emplois collectées depuis {len(COUNTRIES_TO_SCRAPE)} pays")
print(f"{'='*60}")

# Combiner toutes les URLs en une seule liste
job_urls = []
for country_name, urls in all_job_urls.items():
    job_urls.extend(urls)

# Supprimer les doublons (au cas où un même emploi apparaîtrait dans plusieurs pays)
job_urls = list(set(job_urls))
print(f"\n📊 Après suppression des doublons: {len(job_urls)} URLs uniques")


🔍 Collecte des URLs d'emplois depuis plusieurs pays...

Pays à scraper: 🇺🇸 États-Unis, 🇬🇧 Royaume-Uni, 🇨🇦 Canada


🌍 🇺🇸 États-Unis
📄 Scraping page 1...
  ✅ 20 emplois trouvés sur cette page
📄 Scraping page 2...
  ✅ 20 emplois trouvés sur cette page

✅ Total: 40 URLs d'emplois uniques collectées
✅ 40 URLs collectées pour 🇺🇸 États-Unis

🌍 🇬🇧 Royaume-Uni
📄 Scraping page 1...
  ✅ 20 emplois trouvés sur cette page
📄 Scraping page 2...
  ✅ 11 emplois trouvés sur cette page

✅ Total: 31 URLs d'emplois uniques collectées
✅ 31 URLs collectées pour 🇬🇧 Royaume-Uni

🌍 🇨🇦 Canada
📄 Scraping page 1...
  ✅ 20 emplois trouvés sur cette page
📄 Scraping page 2...
  ✅ 16 emplois trouvés sur cette page

✅ Total: 36 URLs d'emplois uniques collectées
✅ 36 URLs collectées pour 🇨🇦 Canada

✅ TOTAL: 107 URLs d'emplois collectées depuis 3 pays

📊 Après suppression des doublons: 107 URLs uniques


## 6. Extraction des Détails de Chaque Emploi

Maintenant, nous extrayons les détails complets de chaque emploi.


In [9]:
# Extraire les détails de chaque emploi
print("\n📋 Extraction des détails des emplois...\n")

all_jobs = []

for i, job_url in enumerate(job_urls[:MAX_JOBS_TO_SCRAPE], 1):
    print(f"  [{i}/{min(MAX_JOBS_TO_SCRAPE, len(job_urls))}] {job_url[:60]}...")
    
    try:
        job_data = extract_job_details_from_aijobs(job_url)
        if job_data and job_data.get('job_title'):
            all_jobs.append(job_data)
            print(f"      ✅ {job_data.get('job_title', 'N/A')[:50]}")
        else:
            print(f"      ⚠️ Données incomplètes")
    except Exception as e:
        print(f"      ❌ Erreur: {e}")
    
    # Délai entre les requêtes
    time.sleep(REQUEST_DELAY)

print(f"\n✅ {len(all_jobs)} emplois extraits avec succès")



📋 Extraction des détails des emplois...

  [1/50] https://aijobs.ai/job/machine-learning-engineer-intern-8...
      ✅ Machine Learning Engineer Intern
  [2/50] https://aijobs.ai/job/ai-software-engineer-2...
      ✅ AI Software Engineer
  [3/50] https://aijobs.ai/job/senior-ai-engineer-ai-labs...
      ✅ Senior AI Engineer, AI Labs
  [4/50] https://aijobs.ai/job/aiml-engineer-solution-designer-toront...
      ✅ AI/ML Engineer / Solution Designer – Toronto - Hyb
  [5/50] https://aijobs.ai/job/machine-learning-engineer-2462...
      ✅ Machine Learning Engineer
  [6/50] https://aijobs.ai/job/sr-machine-learning-engineer-1...
      ✅ Sr. Machine Learning Engineer
  [7/50] https://aijobs.ai/job/backend-software-engineer-python...
      ✅ Backend Software Engineer - Python
  [8/50] https://aijobs.ai/job/senior-applied-ai-engineer-fmd...
      ✅ Senior Applied AI Engineer (f/m/d)
  [9/50] https://aijobs.ai/job/staff-post-silicon-validation-engineer...
      ✅ Staff Post Silicon Validation En

## 7. Traitement et Nettoyage des Données


In [10]:
# Convertir en DataFrame
df = pd.DataFrame(all_jobs)

# Nettoyer les données
if not df.empty:
    # Convertir tech_stack en string pour le CSV (on peut le réanalyser plus tard)
    df['tech_stack_str'] = df['tech_stack'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
    
    # Calculer le salaire moyen si min et max sont disponibles
    df['avg_salary'] = df.apply(
        lambda row: (row['salary_min'] + row['salary_max']) / 2 
        if pd.notna(row['salary_min']) and pd.notna(row['salary_max']) 
        else None, axis=1
    )
    
    # Nettoyer les localisations
    df['location'] = df['location'].fillna('Non spécifiée')
    
    print("📊 Aperçu des données:")
    print(df.head())
    print(f"\n📈 Statistiques:")
    print(f"  - Total d'emplois: {len(df)}")
    print(f"  - Types de contrats: {df['work_mode'].value_counts().to_dict()}")
    print(f"  - Niveaux d'expérience: {df['experience_level'].value_counts().to_dict()}")
    
    if 'avg_salary' in df.columns:
        avg_sal = df['avg_salary'].dropna()
        if len(avg_sal) > 0:
            print(f"  - Salaire moyen: €{avg_sal.mean():,.0f}")
else:
    print("⚠️ Aucune donnée disponible")


📊 Aperçu des données:
                                           job_title  company_name  \
0                   Machine Learning Engineer Intern        Moloco   
1                               AI Software Engineer       Anaplan   
2                        Senior AI Engineer, AI Labs           NPR   
3  AI/ML Engineer / Solution Designer – Toronto -...         Capco   
4                          Machine Learning Engineer  Speechmatics   

                                            location work_mode  \
0                  London,  England,  United Kingdom   On-site   
1                        Manchester,  United Kingdom    Hybrid   
2  Washington,  District of Columbia,  United States    Hybrid   
3                                   Canada - Toronto    Hybrid   
4               Cambridge,  England,  United Kingdom    Hybrid   

  experience_level  salary_min  salary_max  \
0           Junior         NaN         NaN   
1           Senior         NaN         NaN   
2           Senior    

## 8. Analyse des Technologies les Plus Demandées


In [11]:
# Analyser les technologies les plus recherchées
if not df.empty and 'tech_stack' in df.columns:
    all_techs = []
    for tech_list in df['tech_stack']:
        if isinstance(tech_list, list):
            all_techs.extend(tech_list)
    
    tech_counter = Counter(all_techs)
    top_techs = tech_counter.most_common(10)
    
    print("🔧 Top 10 Technologies les Plus Demandées:")
    for tech, count in top_techs:
        print(f"  {tech}: {count} offres")
    
    # Ajouter cette info au DataFrame
    df['top_tech_count'] = df['tech_stack'].apply(
        lambda x: len([t for t in x if t in dict(top_techs[:5])]) if isinstance(x, list) else 0
    )


🔧 Top 10 Technologies les Plus Demandées:
  Python: 34 offres
  Machine Learning: 27 offres
  PyTorch: 17 offres
  TensorFlow: 15 offres
  Go: 11 offres
  AWS: 9 offres
  Java: 8 offres
  Kubernetes: 7 offres
  CI/CD: 7 offres
  Docker: 7 offres


## 9. Sauvegarde des Données


In [12]:
# Préparer le DataFrame pour la sauvegarde
if not df.empty:
    # Sélectionner les colonnes à sauvegarder (inclure country_origin si disponible)
    columns_to_save = [
        'job_title', 'company_name', 'location', 'work_mode', 
        'experience_level', 'salary_min', 'salary_max', 'avg_salary',
        'tech_stack_str', 'job_description', 'job_url'
    ]
    
    # Ajouter country_origin si disponible
    if 'country_origin' in df.columns:
        columns_to_save.insert(3, 'country_origin')  # Après location
    
    # Garder seulement les colonnes qui existent
    save_df = df[[col for col in columns_to_save if col in df.columns]].copy()
    
    # Sauvegarder en CSV
    output_file = 'data/donnees_marche_emploi.csv'
    save_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n💾 Données sauvegardées dans {output_file}")
    print(f"   - {len(save_df)} lignes")
    print(f"   - {len(save_df.columns)} colonnes")
    
    # Afficher la répartition finale par pays
    if 'country_origin' in save_df.columns:
        print(f"\n🌍 Répartition finale par pays:")
        country_final = save_df['country_origin'].value_counts()
        for country, count in country_final.items():
            percentage = (count / len(save_df)) * 100
            print(f"   {country}: {count} offres ({percentage:.1f}%)")
else:
    print("⚠️ Aucune donnée à sauvegarder")



💾 Données sauvegardées dans data/donnees_marche_emploi.csv
   - 50 lignes
   - 11 colonnes
